# LIVRIA : Recommandation des thèmes et des livres 

C'est dans ce notebook -que l'on peut considérer comme le dernier - que mettons en lumières les recommendations de nos différents modèles

### Import des librairies et des données

In [1]:
# Librairies 
import numpy as np 
import pandas as pd
import ipywidgets as wg
from IPython.core.display import Image, display

# Fichiers
ratings = pd.read_csv('data/ratings.csv')
books = pd.read_csv('data/books.csv')
tags = pd.read_csv('data/tags.csv')
btags = pd.read_csv('data/book_tags.csv')

### Affichage des livres

Ici, nous allons crée la fonction permettant d'afficher proprement les livres avec leur page de couverture, le titre l'auteur et la note moyenne. On commence par nettoyer le DataFrame des colonnes d'ont nous n'avons pas besoin ici.

In [2]:
books = books.drop(['goodreads_book_id','best_book_id', 'work_id','books_count','isbn','isbn13','title','language_code','work_ratings_count','work_text_reviews_count','ratings_count','ratings_1','ratings_2','ratings_3','ratings_4','ratings_5','image_url'], axis=1)
books.head()

,book_id,authors,original_publication_year,original_title,average_rating,small_image_url
0,1,Suzanne Collins,2008.0,The Hunger Games,4.34,https://images.gr-assets.com/books/1447303603s...
1,2,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,https://images.gr-assets.com/books/1474154022s...
2,3,Stephenie Meyer,2005.0,Twilight,3.57,https://images.gr-assets.com/books/1361039443s...
3,4,Harper Lee,1960.0,To Kill a Mockingbird,4.25,https://images.gr-assets.com/books/1361975680s...
4,5,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,https://images.gr-assets.com/books/1490528560s...


On crée la fonction qui nous permettra d'afficher les livres :

In [4]:
def show_books(books):    
    print('                   Liste des livres qui pourraient vous plaire\n               ===================================================\n')
    for row in books.itertuples():
        print('{} de {} ({}).     Note moyenne: {}/5'.format(str(row[4]),str(row[2]),int(row[3]),str(row[5])))
        display(Image(url= str(row[6]), width=100, height=100))

In [5]:
# On teste la fonction
livresMontres = books.loc[books['book_id'].isin([1,6,4])]
show_books(livresMontres) # Elle prend en entrée un dataFrame avec les livres que nous souhaitons afficher

                   Liste des livres qui pourraient vous plaire

The Hunger Games de Suzanne Collins (2008).     Note moyenne: 4.34/5


To Kill a Mockingbird de Harper Lee (1960).     Note moyenne: 4.25/5


The Fault in Our Stars de John Green (2012).     Note moyenne: 4.26/5


# Recommandations

## A l'entrée de l'application 

Veuillez répondre au questions ci-dessous :

In [7]:
def get_recommendations(name,sim):
   # indx=df_book_new.loc[df_book_new['title']==name].index
    indx=book_indices[name]
    sim_scores=list(enumerate(sim[indx]))
    new=sorted(sim_scores,key=lambda x: x[1],reverse=True)
   
    new=new[1:11]
    #print(new)
    book_idx=[x[0] for x in new]
    return (df_book_new['title'].iloc[book_idx])

In [13]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
book_id                      10000 non-null int64
authors                      10000 non-null object
original_publication_year    9979 non-null float64
original_title               9415 non-null object
average_rating               10000 non-null float64
ratings_count                10000 non-null int64
small_image_url              10000 non-null object
dtypes: float64(2), int64(2), object(3)
memory usage: 547.0+ KB


In [ ]:
book = books.query("book_id == 11")
Image(url= book['image_url'], width=100, height=100)

In [ ]:
def recommendation(user_id):
    user = book.copy()
    already_read = books_data[books_data['user_id'] == user_id]['book_id'].unique()
    user = user.reset_index()
    user = user[~user['book_id'].isin(already_read)]
    user['Estimate_Score']=user['book_id'].apply(lambda x: algo.predict(user_id, x).est)
    user = user.drop('book_id', axis = 1)
    user = user.sort_values('Estimate_Score', ascending=False)
    print(user.head(10))